In [ ]:
!pip install tensorflow

  Using cached tensorflow-2.12.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (585.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached grpcio-1.54.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.1 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached wrapt-1.14.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (81 kB)
  Using cached tensorflow_io_gcs_filesystem-0.32.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached protobuf-4.22.3-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
  Using cached tensorboard-2.12.2-py3-none-any.whl (5.6 MB)
  Using cached flatbuffers-23.3.3-py2.py3-none-any.whl (26 kB)
     |████████████████████████████████| 17.1 MB 2.4 MB/s eta 0:00:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl 

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
%cd Major2

/content/gdrive/MyDrive/Major2


In [ ]:
%ls

test/  train/  val/


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [ ]:
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

In [ ]:
batch_size = 32
img_width, img_height = 224, 224
num_classes = 2

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height),
                                                    batch_size=batch_size, class_mode='categorical')

Found 1921 images belonging to 2 classes.


In [ ]:
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(img_width, img_height),
                                                batch_size=batch_size, class_mode='categorical')

Found 549 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(img_width, img_height),
                                                  batch_size=batch_size, class_mode='categorical')

Found 275 images belonging to 2 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 2/10
61/61 [==============================] - 61s 999ms/step - loss: 0.6237 - accuracy: 0.6450 - val_loss: 0.5982 - val_accuracy: 0.6630
Epoch 3/10
61/61 [==============================] - 61s 1s/step - loss: 0.5753 - accuracy: 0.6866 - val_loss: 0.5299 - val_accuracy: 0.7250
Epoch 4/10
61/61 [==============================] - 61s 1s/step - loss: 0.5690 - accuracy: 0.6950 - val_loss: 0.5412 - val_accuracy: 0.6885
Epoch 5/10
61/61 [==============================] - 63s 1s/step - loss: 0.5299 - accuracy: 0.7267 - val_loss: 0.5269 - val_accuracy: 0.7140
Epoch 6/10
61/61 [==============================] - 61s 999ms/step - loss: 0.5053 - accuracy: 0.7434 - val_loss: 0.5025 - val_accuracy: 0.7450
Epoch 7/10
61/61 [==============================] - 61s 1s/step - loss: 0.5052 - accuracy: 0.7361 - val_loss: 0.5388 - val_accuracy: 0.6885
Epoch 8/10
61/61 [==============================] - 61s 1s/step - loss: 0.4791 - accuracy: 0.7527 - val_loss: 0.4962 - val_accuracy: 0.7687
Epoch 9/10
61/

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

9/9 [==============================] - 152s 19s/step - loss: 0.5069 - accuracy: 0.7382
Test accuracy: 0.7381818294525146


In [ ]:
val_loss, val_acc = model.evaluate(val_generator)
print('Val accuracy:', val_acc)

18/18 [==============================] - 10s 542ms/step - loss: 0.4986 - accuracy: 0.7632
Val accuracy: 0.7632058262825012


In [ ]:
model.save('models/clickbait_classifier.h5')